# 5. Ensembles de Arboles de Decision

## 5.4 GBDT LightGBM

La técnica de Gradient Boosting fue creada por Jerome H. Friedman en 1999 - 2001
<br>Se implementaron librerías ineficientes
<br>En 2016 se crea XGBoost, en 2017 LightGBM

El Gradient Boosting of Decision Trees es un ensemble de árboles de decisión, para un nuevo registro la predicción se hace sumando el score que cada arbol asigna a ese registro.

En GBDT la construccion de los árboles es secuencial, ya que el arbol n-simo se genera para predecir el error del modelo conformado por los  n-1 arboles previos, aunque sea un arbol de clasificación lo que se predice es un numero real mediante un arbol de regresión.


<br>Qué tipo de perturbaciones se realiza LightGBM

*   Se perturba el dataset, seleccionando para cada arbol un subconjunto de las columnas.
*   El algortimo de arbol de decisión no presenta perturbaciones

Cada arbolito de LightGBM se entrena sobre un dataset perturbado, que en principio posee :
* todos los registros del dataset original
* solo un porcentaje *feature_fraction* de las columnas originales del dataset

#### 5.4.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/labo1r/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


### 5.4.2  LightGBM, una corrida

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,649923,34.8,1439354,76.9,1239014,66.2
Vcells,1209893,9.3,8388608,64.0,1924955,14.7


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

Loading required package: data.table

Loading required package: rpart

Loading required package: rlist

Loading required package: lightgbm



Aqui debe cargar SU semilla primigenia

In [3]:
PARAM <- list()
PARAM$experimento <- 6001
PARAM$semilla_primigenia <- 532981

# estos hiperparametros de LightGBM surgieron de una Bayesian Optimization
PARAM$lgb$num_iterations <- 666  # cantidad de arbolitos
PARAM$lgb$learning_rate <- 0.01
PARAM$lgb$feature_fraction <- 0.44
PARAM$lgb$min_data_in_leaf <- 166
PARAM$lgb$num_leaves <- 266
PARAM$lgb$max_bin <- 888


In [4]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("KA", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [5]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv", stringsAsFactors = TRUE)

In [6]:
# paso la clase a binaria que tome valores {0,1}  enteros
# set trabaja con la clase  POS = { BAJA+1, BAJA+2 }
# esta estrategia es MUY importante
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2", "BAJA+1"), 1L, 0L)]

In [7]:
# los campos que se van a utilizar
campos_buenos <- setdiff(colnames(dataset), c("clase_ternaria", "clase01"))

In [8]:
# establezco donde entreno
dataset[, train := 0L]
dataset[foto_mes %in% c(202107), train := 1L]

In [9]:
# dejo los datos en el formato que necesita LightGBM
dtrain <- lgb.Dataset(
  data= data.matrix(dataset[train == 1L, campos_buenos, with = FALSE]),
  label= dataset[train == 1L, clase01]
)

In [10]:
# genero el modelo
# estos hiperparametros  salieron de una laaarga Optmizacion Bayesiana
set.seed( PARAM$semilla_primigenia ) # Establezco la semilla aleatoria

modelo <- lgb.train(
  data= dtrain,
  param= list(
    objective= "binary",
    max_bin= PARAM$lgb$max_bin,
    learning_rate= PARAM$lgb$learning_rate,
    num_iterations= PARAM$lgb$num_iterations,
    num_leaves= PARAM$lgb$num_leaves,
    min_data_in_leaf= PARAM$lgb$min_data_in_leaf,
    feature_fraction= PARAM$lgb$feature_fraction,
    seed= PARAM$semilla_primigenia
  )
)


[LightGBM] [Info] Number of positive: 2385, number of negative: 162211
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034909 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 56685
[LightGBM] [Info] Number of data points in the train set: 164596, number of used features: 150
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.014490 -> initscore=-4.219699
[LightGBM] [Info] Start training from score -4.219699


In [12]:
# ahora imprimo la importancia de variables
tb_importancia <- as.data.table(lgb.importance(modelo))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file = archivo_importancia,
  sep = "\t"
)


In [13]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(modelo, "modelo.txt" )

In [14]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes == 202109]

# aplico el modelo a los datos nuevos
prediccion <- predict(
  modelo,
  data.matrix(dfuture[, campos_buenos, with = FALSE])
)


In [15]:
# tabla de prediccion

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file = "prediccion.txt",
  sep = "\t"
)

In [16]:
# subidas a Kaggle
# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

In [17]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

cortes <- seq(9000, 13500, by = 500)

for (envios in cortes) {

  tb_prediccion[, Predicted := 0L]
  tb_prediccion[1:envios, Predicted := 1L]

  archivo_kaggle <- paste0("KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file = archivo_kaggle,
    sep = ","
  )

  # subida a Kaggle
  comando <- "kaggle competitions submit"
  competencia <- "-c labo-i-2025-rosario"
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'num_iterations=", PARAM$lgb$num_iterations,
    "  learning_rate=", PARAM$lgb$learning_rate,
    "  feature_fraction=", PARAM$lgb$feature_fraction,
    "  min_data_in_leaf=", PARAM$lgb$min_data_in_leaf,
    "  num_leaves=",PARAM$lgb$num_leaves,
    "  max_bin=", PARAM$lgb$max_bin,
  "'" )

  linea <- paste( comando, competencia, arch, mensaje)
  salida <- system(linea, intern=TRUE)
  cat(salida)
}

Successfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 Rosario



---



### 5.4.3  LightGBM  optimizacion de hiperparámetros

La optimizacion de los hiperparámetros de LightGBM mediante el método de optimizacion bayesiana será su *caballito de batalla* durante la asignatura !

limpio el ambiente de R

In [18]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1817914,97.1,6921546,369.7,10814915,577.6
Vcells,3567507,27.3,114228135,871.5,135671748,1035.1


In [19]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")


Loading required package: parallel

Loading required package: primes

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames




Aqui debe cargar SU semilla primigenia

In [20]:
PARAM <- list()
PARAM$experimento <- 5430
PARAM$semilla_primigenia <- 532981

# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos
PARAM$trainingstrategy$undersampling <- 1.0

PARAM$hyperparametertuning$iteraciones <- 100
PARAM$hyperparametertuning$xval_folds <- 5
PARAM$hyperparametertuning$POS_ganancia <- 117000
PARAM$hyperparametertuning$NEG_ganancia <- -3000

# Aqui se cargan los bordes de los hiperparametros
PARAM$hs <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.01, upper = 0.05),
  makeIntegerParam("num_leaves", lower = 4L, upper = 32L),
  makeNumericParam("feature_fraction", lower = 0.6, upper = 1.0),
  makeNumericParam("bagging_fraction", lower = 0.6, upper = 1.0),
  makeIntegerParam("bagging_freq", lower = 1L, upper = 10L),
  makeIntegerParam("min_data_in_leaf", lower = 50L, upper = 150L),
  makeNumericParam("lambda_l1", lower = 0.0, upper = 1.0),
  makeNumericParam("lambda_l2", lower = 0.0, upper = 1.0),
  makeIntegerParam("max_bin", lower = 15L, upper = 64L),
  makeIntegerParam("envios", lower = 8000L, upper = 13000L)
)




In [21]:
# graba a un archivo los componentes de lista
# para el primer registro, escribe antes los titulos

loguear <- function(
    reg, arch = NA, folder = "./work/",
    ext = ".txt", verbose = TRUE) {

  archivo <- arch
  if (is.na(arch)) archivo <- paste0(folder, substitute(reg), ext)

  if (!file.exists(archivo)) # Escribo los titulos
    {
      linea <- paste0(
        "fecha\t",
        paste(list.names(reg), collapse = "\t"), "\n"
      )

      cat(linea, file = archivo)
    }

  linea <- paste0(
    format(Sys.time(), "%Y%m%d %H%M%S"), "\t", # la fecha y hora
    gsub(", ", "\t", toString(reg)), "\n"
  )

  cat(linea, file = archivo, append = TRUE) # grabo al archivo

  if (verbose) cat(linea) # imprimo por pantalla
}


In [22]:
# esta funcion calcula internamente la ganancia de la prediccion probs
# es llamada por lightgbm luego de construir cada  arbolito

fganancia_logistic_lightgbm <- function(probs, datos) {
  vpesos <- get_field(datos, "weight")

  # vector de ganancias
  vgan <- ifelse(vpesos == 1.0000002, PARAM$hyperparametertuning$POS_ganancia,
    ifelse(vpesos == 1.0000001, PARAM$hyperparametertuning$NEG_ganancia,
      PARAM$hyperparametertuning$NEG_ganancia /
        PARAM$trainingstrategy$undersampling
    )
  )

  tbl <- as.data.table(list("vprobs" = probs, "vgan" = vgan))
  setorder(tbl, -vprobs)
  ganancia <- tbl[1:GLOBAL_envios, sum(vgan)]

  return(list(
    "name" = "ganancia",
    "value" = ganancia,
    "higher_better" = TRUE
  ))
}


In [23]:
# esta funcion solo puede recibir los parametros que se estan optimizando
# el resto de los parametros se pasan como variables globales,
# la semilla del mal ...


EstimarGanancia_lightgbm <- function(x) {
  gc() # libero memoria

  # llevo el registro de la iteracion por la que voy
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  # para usar en fganancia_logistic_lightgbm
  # asigno la variable global
  GLOBAL_envios <<- as.integer(x$envios / PARAM$hyperparametertuning$xval_folds)

  # cantidad de folds para cross validation
  kfolds <- PARAM$hyperparametertuning$xval_folds

  param_basicos <- list(
    objective= "binary",
    metric= "custom",
    first_metric_only= TRUE,
    boost_from_average= TRUE,
    feature_pre_filter= FALSE,
    verbosity= -100,
    max_bin= 31, # por ahora, lo dejo fijo
    num_iterations= 9999, # valor grande, lo limita early_stopping_rounds
    force_row_wise= TRUE, # para evitar warning
    seed= ksemilla_azar1
  )

  # el parametro discolo, que depende de otro
  param_variable <- list(
    early_stopping_rounds =
      as.integer(50 + 5 / x$learning_rate)
  )

  param_completo <- c(param_basicos, param_variable, x)

  set.seed(ksemilla_azar1)
  modelocv <- lgb.cv(
    data= dtrain,
    eval= fganancia_logistic_lightgbm,
    stratified= TRUE, # sobre el cross validation
    nfold= kfolds, # folds del cross validation
    param= param_completo,
    verbose= -100
  )

  # obtengo la ganancia
  ganancia <- unlist(modelocv$record_evals$valid$ganancia$eval)[modelocv$best_iter]

  ganancia_normalizada <- ganancia * kfolds # normailizo la ganancia

  # asigno el mejor num_iterations
  param_completo$num_iterations <- modelocv$best_iter
  # elimino de la lista el componente
  param_completo["early_stopping_rounds"] <- NULL


  # el lenguaje R permite asignarle ATRIBUTOS a cualquier variable
  # esta es la forma de devolver un parametro extra
  attr(ganancia_normalizada, "extras") <-
    list("num_iterations" = modelocv$best_iter)

  # logueo
  xx <- param_completo
  xx$ganancia <- ganancia_normalizada # le agrego la ganancia
  xx$iteracion <- GLOBAL_iteracion
  loguear(xx, arch = klog)

  # Voy registrando la importancia de variables
  if (ganancia_normalizada > GLOBAL_gananciamax) {
    GLOBAL_gananciamax <<- ganancia_normalizada
    modelo <- lgb.train(
      data = dtrain,
      param = param_completo,
      verbose = -100
    )

    tb_importancia <- as.data.table(lgb.importance(modelo))
    archivo_importancia <- paste0("impo_", GLOBAL_iteracion, ".txt")
    fwrite(tb_importancia,
      file = archivo_importancia,
      sep = "\t" )

    loguear(xx, arch = klog_mejor)
  }

  return(ganancia_normalizada)
}


aqui se inicia el programa

In [24]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [25]:
# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, 2 )
ksemilla_azar1 <- PARAM$semillas[1]
ksemilla_azar2 <- PARAM$semillas[2]


In [26]:
# en estos archivos quedan los resultados

kbayesiana <- paste0(PARAM$experimento, ".RDATA")
klog <- paste0(PARAM$experimento, ".txt")
klog_mejor <- paste0(PARAM$experimento, "_mejor.txt")

GLOBAL_iteracion <- 0 # inicializo la variable global
GLOBAL_gananciamax <- -1 # inicializo la variable global

# si ya existe el archivo log, traigo hasta donde llegue
if (file.exists(klog)) {
  tabla_log <- fread(klog)
  GLOBAL_iteracion <- nrow(tabla_log)
  GLOBAL_gananciamax <- tabla_log[, max(ganancia)]
}


In [27]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [28]:
dataset <- dataset[foto_mes %in% c(202107)]

In [29]:
# paso la clase a binaria que tome valores {0,1}  enteros
dataset[
  foto_mes %in% c(202107),
  clase01 := ifelse(clase_ternaria == "CONTINUA", 0L, 1L)
]

In [30]:
# los campos que se van a utilizar
campos_buenos <- setdiff(
  colnames(dataset),
  c("clase_ternaria", "clase01", "azar", "training")
)


In [31]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

set.seed(ksemilla_azar2)
dataset[, azar := runif(nrow(dataset))]
dataset[, training := 0L]
dataset[
  foto_mes %in% c(202107) &
    (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

In [32]:
# dejo los datos en el formato que necesita LightGBM
dtrain <- lgb.Dataset(
  data = data.matrix(dataset[training == 1L, campos_buenos, with = FALSE]),
  label = dataset[training == 1L, clase01],
  weight = dataset[training == 1L, ifelse(clase_ternaria == "BAJA+2", 1.0000002, ifelse(clase_ternaria == "BAJA+1", 1.0000001, 1.0))],
  free_raw_data = FALSE
)


In [33]:
# Aqui comienza la configuracion de la Bayesian Optimization
funcion_optimizar <- EstimarGanancia_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
  fn = funcion_optimizar, # la funcion que voy a maximizar
  minimize = FALSE, # estoy Maximizando la ganancia
  noisy = TRUE,
  par.set = PARAM$hs, # definido al comienzo del programa
  has.simple.signature = FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time = 600, # se graba cada 600 segundos
  save.file.path = kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters = PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type = "se",
  covtype = "matern3_2",
  control = list(trace = TRUE)
)


In [34]:
# inicio la optimizacion bayesiana

if (!file.exists(kbayesiana)) {
  run <- mbo(obj.fun, learner = surr.km, control = ctrl)
} else {
  run <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

20250604 213445	binary	custom	TRUE	TRUE	FALSE	-100	31	764	TRUE	974849	0.020467091150037	12	0.911601501569353	146	26	12227	59565000	63


[mbo] 33: learning_rate=0.0205; num_leaves=12; feature_fraction=0.912; min_data_in_leaf=146; max_bin=26; envios=12227 : y = 5.96e+07 : 84.1 secs : infill_ei

Saved the current state after iteration 34 in the file 5430.RDATA.



20250604 213557	binary	custom	TRUE	TRUE	FALSE	-100	31	497	TRUE	974849	0.0204942848933191	23	0.962774241777956	113	51	12448	61185000	64


[mbo] 34: learning_rate=0.0205; num_leaves=23; feature_fraction=0.963; min_data_in_leaf=113; max_bin=51; envios=12448 : y = 6.12e+07 : 64.9 secs : infill_ei



20250604 213631	binary	custom	TRUE	TRUE	FALSE	-100	31	233	TRUE	974849	0.0421366810318537	23	0.963457562322754	150	26	12036	61095000	65


...

[mbo] 98: learning_rate=0.0167; num_leaves=6; feature_fraction=0.635; min_data_in_leaf=65; max_bin=63; envios=10318 : y = 5.8e+07 : 69.5 secs : infill_ei



20250604 223405	binary	custom	TRUE	TRUE	FALSE	-100	31	251	TRUE	974849	0.0365085974028986	28	0.795693533823918	137	20	8567	60345000	129


[mbo] 99: learning_rate=0.0365; num_leaves=28; feature_fraction=0.796; min_data_in_leaf=137; max_bin=20; envios=8567 : y = 6.03e+07 : 37.2 secs : infill_ei



20250604 223456	binary	custom	TRUE	TRUE	FALSE	-100	31	391	TRUE	974849	0.0308397483822703	30	0.94847354055671	57	58	10864	61140000	130


[mbo] 100: learning_rate=0.0308; num_leaves=30; feature_fraction=0.948; min_data_in_leaf=57; max_bin=58; envios=10864 : y = 6.11e+07 : 49.7 secs : infill_ei

Saved the final state in the file 5430.RDATA





---

